In [1]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.sports_betting_capstone
nfl_lines = db.nfl_lines

In [38]:
nfl_rows = nfl_lines.find()
nfl_pddf = pd.DataFrame(list(nfl_rows))
nfl_pddf

,_id,Team,Win-Loss,Streak,Open,Closing,Consensus_Spread,Consensus_Money,Consensus_OU,Season,Week
0,5f98877a1e3e51b1b60328e4,451 Houston,(0-0) (0-0 A),,56.5,53.5,50%,26%,72%,2020,1
1,5f98877a1e3e51b1b60328e5,452 Kansas City Â«,(0-0) (0-0 H),,-10,-9.5,50%,74%,28%,2020,1
2,5f98877a1e3e51b1b60328e6,0 Las Vegas Â«,(0-0) (0-0 A),,,,,,,2020,1
3,5f98877a1e3e51b1b60328e7,0 Carolina,(0-0) (0-0 H),,,,,,,2020,1
4,5f98877a1e3e51b1b60328e8,453 Miami,(0-0) (0-0 A),,43.5,41.5,42%,51%,58%,2020,1
...,...,...,...,...,...,...,...,...,...,...,...
2339,5f98883cc01cbbbbde303746,314 New England Â«,(14-2) (6-2 H),W-8,,-5.5,55%,40%,34%,2016,20
2340,5f98883dc01cbbbbde303748,497 AFC,(0-0) (0-0 A),,68.5,80,34%,69%,77%,2016,21
2341,5f98883dc01cbbbbde303749,498 NFC,(0-0) (0-0 H),,-4.5,-4,66%,31%,23%,2016,21
2342,5f98883ec01cbbbbde30374b,101 New England,(14-2) (8-0 A),W-9,PK,-3,63%,47%,52%,2016,22


In [39]:
for idx, row in nfl_pddf.iterrows():
    if idx == 0 or idx % 2 == 0:
        nfl_pddf.loc[idx, 'Include'] = True
        nfl_pddf.loc[idx, 'Team 1'] = nfl_pddf.loc[idx, 'Team']
        nfl_pddf.loc[idx, 'Team 2'] = nfl_pddf.loc[idx + 1, 'Team']
        nfl_pddf.loc[idx, 'Spread Open'] = nfl_pddf.loc[idx , 'Open']
        nfl_pddf.loc[idx, 'Spread Close'] = nfl_pddf.loc[idx, 'Closing']
        nfl_pddf.loc[idx, 'Spread Consensus Team 1'] = nfl_pddf.loc[idx, 'Consensus_Spread']
        nfl_pddf.loc[idx, 'Spread Consensus Team 2'] = nfl_pddf.loc[idx + 1, 'Consensus_Spread']
        nfl_pddf.loc[idx, 'O/U Open'] = nfl_pddf.loc[idx , 'Open']
        nfl_pddf.loc[idx, 'O/U Close'] = nfl_pddf.loc[idx, 'Closing']
        nfl_pddf.loc[idx, 'Over Consensus'] = nfl_pddf.loc[idx, 'Consensus_OU']
        nfl_pddf.loc[idx, 'Under Consensus'] = nfl_pddf.loc[idx + 1, 'Consensus_OU']
        nfl_pddf.loc[idx, 'Moneyline Consensus Team 1'] = nfl_pddf.loc[idx, 'Consensus_Money']
        nfl_pddf.loc[idx, 'Moneyline Consensus Team 2'] = nfl_pddf.loc[idx + 1, 'Consensus_Money']
    else:
        nfl_pddf.loc[idx, 'Include'] = False

In [46]:
clean_nfl_pddf = nfl_pddf.drop(nfl_pddf.columns[range(9)], axis = 1)[nfl_pddf['Include']].reset_index()

In [47]:
clean_nfl_pddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   index                       1172 non-null   int64 
 1   Season                      1172 non-null   int64 
 2   Week                        1172 non-null   int64 
 3   Include                     1172 non-null   object
 4   Team 1                      1172 non-null   object
 5   Team 2                      1172 non-null   object
 6   Spread Open                 1172 non-null   object
 7   Spread Close                1172 non-null   object
 8   Spread Consensus Team 1     1172 non-null   object
 9   Spread Consensus Team 2     1172 non-null   object
 10  O/U Open                    1172 non-null   object
 11  O/U Close                   1172 non-null   object
 12  Over Consensus              1172 non-null   object
 13  Under Consensus             1172 non-null   obje

In [55]:
clean_nfl_pddf['Team 1'] = clean_nfl_pddf['Team 1'].apply(lambda x: ''.join(filter(str.isalpha, x)))

In [56]:
clean_nfl_pddf

,index,Season,Week,Include,Team 1,Team 2,Spread Open,Spread Close,Spread Consensus Team 1,Spread Consensus Team 2,O/U Open,O/U Close,Over Consensus,Under Consensus,Moneyline Consensus Team 1,Moneyline Consensus Team 2
0,0,2020,1,True,Houston,452 Kansas City Â«,56.5,53.5,50%,50%,56.5,53.5,72%,28%,26%,74%
1,2,2020,1,True,LasVegasÂ,0 Carolina,,,,,,,,,,
2,4,2020,1,True,Miami,454 New England Â«,43.5,41.5,42%,58%,43.5,41.5,58%,42%,51%,49%
3,6,2020,1,True,Cleveland,456 Baltimore Â«,48.5,47.5,40%,60%,48.5,47.5,72%,28%,25%,75%
4,8,2020,1,True,NYJets,458 Buffalo Â«,40.0,39,20%,80%,40.0,39,49%,51%,23%,77%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,2334,2016,19,True,GreenBayÂ,308 Dallas,51.5,53.5,56%,44%,51.5,53.5,76%,24%,77%,23%
1168,2336,2016,20,True,GreenBay,312 Atlanta Â«,61,61,65%,35%,61,61,63%,37%,72%,28%
1169,2338,2016,20,True,Pittsburgh,314 New England Â«,,49.5,45%,55%,,49.5,66%,34%,60%,40%
1170,2340,2016,21,True,AFC,498 NFC,68.5,80,34%,66%,68.5,80,77%,23%,69%,31%
